In [ ]:
import json
import os
import sys
import tarfile

from pdbtools import *
import requests
from datetime import datetime
from pathlib import Path

import tengu

### 0) Setup

In [ ]:
# Set our token - ensure you have exported TENGU_TOKEN in your shell; or just replace the os.getenv with your token
TOKEN = os.getenv("TENGU_TOKEN")
# You might have a custom deployment url, by default it will use https://tengu.qdx.ai
URL = os.getenv("TENGU_URL") or "https://tengu.qdx.ai"

In [ ]:
# Define our project information
DESCRIPTION = "tengu-py demo notebook"
TAGS = ["qdx", "tengu-py", "demo", "hermes"]
WORK_DIR = Path.home() / "qdx" / "tengu-py-hermes"
OUT_DIR = WORK_DIR / "runs"
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Set our inputs
PROTEIN_PDB_PATH = WORK_DIR / "test_P.pdb"
LIGAND_SDF_PATH = WORK_DIR / "test_L.sdf"

In [ ]:
# Get our client, for calling modules and using the tengu API
client = tengu.Provider(access_token=TOKEN, url=URL)

In [ ]:
# Get our latest modules as a dict[module_name, module_path]
modules = client.get_latest_module_paths()

In [ ]:
convert_protein = client.run2(
    modules["convert"],
    [
        "PDB",
        PROTEIN_PDB_PATH,
    ],
    target="NIX",
    resources={"storage": 1_024_000_000, "walltime": 15},
    tags=TAGS,
)
convert_protein_run_id = convert_protein["module_instance_id"]
converted_protein = convert_protein["output_ids"][0]

pick_protein = client.run2(
        modules["pick_conformer"],
    [
        converted_protein,
        0
    ]
)
picked_protein = pick_protein["output_ids"][0]

print(f"{datetime.now().time()} | Running protein conversion!")

In [ ]:
fragment_protein = client.run2(
    modules["fragment_aa"],
    [
        picked_protein,
        1,
        "All"
    ],
    target="NIX",
    resources={"storage": 1_024_000_000, "walltime": 15},
)

In [ ]:
client.poll_module_instance(fragment_protein["module_instance_id"])

In [ ]:
convert_ligand = client.run2(
    modules["convert"],
    [
        "SDF",
        LIGAND_SDF_PATH,
    ],
    target="NIX",
    resources={"storage": 1_024_000_000, "walltime": 15},
    tags=TAGS,
)
convert_ligand_run_id = convert_ligand["module_instance_id"]
converted_ligand = convert_ligand["output_ids"][0]

pick_ligand = client.run2(
        modules["pick_conformer"],
    [
        converted_ligand,
        0
    ]
)
picked_ligand = pick_ligand["output_ids"][0]
        

print(f"{datetime.now().time()} | Running ligand conversion!")

In [ ]:
hermes_lattice = client.run2(
    modules["hermes_lattice"],
    [
        fragment_protein["output_ids"][0],
        picked_ligand,
        None,
        None,
    ],
    target="NIX_SSH_3",
    resources={"storage": 1_024_000_000, "walltime": 15},
    tags=TAGS,
)

In [ ]:
client.module_instance("d2d966ce-d55e-449a-937a-fb83cc38c1de")